# Chapter 8: Data Wrangling: Join, Combine & Reshape

## 8.1 Hierarchical Indexing

Hierarchical indexing is an important feature of pandas that enables you to have mul‐ tiple (two or more) index levels on an axis. Somewhat abstractly, it provides a way for you to work with higher dimensional data in a lower dimensional form. Let’s start with a simple example; create a Series with a list of lists (or arrays) as the index:

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [6]:
data = pd.Series(np.random.randn(9), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1,2,3,1,3,1,2,2,3]])

In [7]:
data

a  1    0.597924
   2    0.598636
   3    0.242738
b  1   -0.027822
   3    1.996268
c  1    0.433442
   2    1.601453
d  2   -2.139175
   3   -1.135932
dtype: float64

<b>What you’re seeing is a prettified view of a Series with a MultiIndex as its index. The “gaps” in the index display mean “use the label directly above”:

In [8]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

<b>With a hierarchically indexed object, so-called partial indexing is possible, enabling
you to concisely select subsets of the data:

In [9]:
data['b']

1   -0.027822
3    1.996268
dtype: float64

In [10]:
data['b':'c']

b  1   -0.027822
   3    1.996268
c  1    0.433442
   2    1.601453
dtype: float64

In [11]:
data.loc[['b', 'd']]

b  1   -0.027822
   3    1.996268
d  2   -2.139175
   3   -1.135932
dtype: float64

In [12]:
data.loc[:, 2]

a    0.598636
c    1.601453
d   -2.139175
dtype: float64

<b>Hierarchical indexing plays an important role in reshaping data and group-based operations like forming a pivot table. For example, you could rearrange the data into a DataFrame using its unstack method:

In [13]:
data.unstack()

,1,2,3
a,0.597924,0.598636,0.242738
b,-0.027822,NaN,1.996268
c,0.433442,1.601453,NaN
d,NaN,-2.139175,-1.135932


<b>The inverse operation of unstack is stack:

In [14]:
data.unstack().stack()

a  1    0.597924
   2    0.598636
   3    0.242738
b  1   -0.027822
   3    1.996268
c  1    0.433442
   2    1.601453
d  2   -2.139175
   3   -1.135932
dtype: float64

<b>With a DataFrame, either axis can have a hierarchical index:

In [15]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

In [16]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

<b>The hierarchical levels can have names (as strings or any Python objects). If so, these will show up in the console output:

In [17]:
frame.index.names = ['key1', 'key2']

In [18]:
frame.columns.names = ['state', 'color']

In [19]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

<b>With partial column indexing you can similarly select groups of columns:

In [20]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

<b>A MultiIndex can be created by itself and then reused; the columns in the preceding DataFrame with level names could be created like this:

```
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['state', 'color'])
```

## Reordering and Sorting Levels

At times you will need to rearrange the order of the levels on an axis or sort the data by the values in one specific level. The swaplevel takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise unaltered):

In [23]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

sort_index, on the other hand, sorts the data using only the values in a single level. When swapping levels, it’s not uncommon to also use sort_index so that the result is lexicographically sorted by the indicated level:

In [24]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [25]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## Summary Statistics by Level

In [26]:
frame.sum(level='key2')

/var/folders/5v/j_knnv350nsfj2p76j0_nrcm0000gq/T/ipykernel_6070/2004046222.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='key2')


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [27]:
frame.sum(level='color', axis=1)

/var/folders/5v/j_knnv350nsfj2p76j0_nrcm0000gq/T/ipykernel_6070/4133796543.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='color', axis=1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Indexing with a DataFrame's columns

<b>It’s not unusual to want to use one or more columns from a DataFrame as the row index; alternatively, you may wish to move the row index into the DataFrame’s col‐ umns. Here’s an example DataFrame:

In [28]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1), 'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
'd':[0,1,2,0,1,2,3]})

In [29]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


<b>DataFrame’s set_index function will create a new DataFrame using one or more of its columns as the index:
  

In [30]:
frame2 = frame.set_index(['c', 'd'])

In [31]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

<b>By default the columns are removed from the DataFrame, though you can leave them in:

In [32]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

<b>reset_index, on the other hand, does the opposite of set_index; the hierarchical index levels are moved into the columns:

In [33]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 Combining and Merging Datasets